In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer
# import libraries
import boto3, re, sys, math, json, os, sagemaker, urllib.request, random
from sagemaker import get_execution_role
import numpy as np                                
import pandas as pd                               
import matplotlib.pyplot as plt                   
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime                 
from sagemaker.predictor import csv_serializer   
from pandas.api.types import CategoricalDtype
from sklearn.model_selection import train_test_split

In [13]:
# Define IAM role
role = get_execution_role()
prefix = 'xgboost'
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest'} # each region has its XGBoost container
my_region = boto3.session.Session().region_name # set the region of the instance

print("Success - the MySageMakerInstance is in the " + my_region + " region. You will use the " + containers[my_region] + " container for your SageMaker endpoint.")


Success - the MySageMakerInstance is in the us-west-2 region. You will use the 433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest container for your SageMaker endpoint.


In [22]:
bucket_name = 'hcp-ml-project-tune-weight' # <--- change this variable to a unique name for your bucket
bucket = bucket_name
s3 = boto3.resource('s3')
try:
    if  my_region == 'us-east-1':
      s3.create_bucket(Bucket=bucket_name)
    else: 
      s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
    print('S3 bucket created successfully')
except Exception as e:
    print('S3 error: ',e)


S3 bucket created successfully


In [15]:
#split the downloaded data into train/test/validation files
FILE_TRAIN = 'train'
FILE_VALIDATION = 'validation'
FILE_TEST = 'test'
columns = ['WILL_ATTACH_IN_60', 'ORGANIZATION_ID', 
       'PLAN_TIER', 'BILLING_PLAN_ID', 'CC_FEE', 'ASP', 'INDUSTRY', 'ORGSIZE',
       'BANK_CONNECTED', 'BANK_VERIFIED', 'CREATED', 'SCHEDULED', 'OMW',
       'STARTED', 'IN_PROGRESS', 'ESTIMATED', 'INVOICED', 'TOTALPAYMENT_COUNT',
       'TOTALPAYMENT_AMOUNT', 'ONLINE_BOOKINGS', 'CC_COUNT', 'CC_AMOUNT',
       'INSTAPAY_AMOUNT', 'ACH_COUNT', 'CASH_COUNT', 'CASH_AMOUNT',
       'CHECK_COUNT', 'CHECK_AMOUNT', 'OTHER_COUNT', 'DAYS_SINCE_ENROLLMENT',
       'REASON_TO_ADOPT', 'STAGENAME_PRODUCT', 'CC_COUNT_ALL',
       'AVG_TIME_TO_PAYMENT', 'COMMITMENTS_ALL', 'COMMITMENTS', 'CC_PERC',
       'CASH_PERC', 'CHECK_PERC', 'OTHER_PERC', 'ACH_PERC']
columns_new = ['WILL_ATTACH_IN_60', 'weight', 'ORGANIZATION_ID', 
       'PLAN_TIER', 'BILLING_PLAN_ID', 'CC_FEE', 'ASP', 'INDUSTRY', 'ORGSIZE',
       'BANK_CONNECTED', 'BANK_VERIFIED', 'CREATED', 'SCHEDULED', 'OMW',
       'STARTED', 'IN_PROGRESS', 'ESTIMATED', 'INVOICED', 'TOTALPAYMENT_COUNT',
       'TOTALPAYMENT_AMOUNT', 'ONLINE_BOOKINGS', 'CC_AMOUNT',
       'INSTAPAY_AMOUNT', 'ACH_COUNT', 'CASH_COUNT', 'CASH_AMOUNT',
       'CHECK_COUNT', 'CHECK_AMOUNT', 'OTHER_COUNT', 'DAYS_SINCE_ENROLLMENT',
       'REASON_TO_ADOPT', 'STAGENAME_PRODUCT', 'CC_COUNT_ALL',
       'AVG_TIME_TO_PAYMENT', 'COMMITMENTS_ALL', 'COMMITMENTS', 'CC_PERC',
       'CASH_PERC', 'CHECK_PERC', 'OTHER_PERC', 'ACH_PERC']


In [27]:
def train_test_group_by_id(df, perc):
    groups = df.groupby('ORGANIZATION_ID')

    split_perc = int(np.rint(df.shape[0] * perc))
    train_groups = set()
    test_groups = set()
    train_rows = 0
    for name, group in groups:
        train_rows += group.shape[0]
        train_groups.add(name)
        if train_rows > split_perc:
            break
    train = groups.filter(lambda x: x.name in train_groups)
    test = groups.filter(lambda x: x.name not in train_groups)
    s1 = set(train['ORGANIZATION_ID'].unique())
    s2 = set(test['ORGANIZATION_ID'].unique())
    print(len(s1.intersection(s2)))
    print(train_rows, split_perc)
    print(train.shape, test.shape)
    return train, test

def assign_weight(row):
    attach_label = row['WILL_ATTACH_IN_60']
    cc_count = row['CC_COUNT']
    if attach_label == 1 and cc_count > 0:
        return 1
    elif attach_label == 0 and cc_count <= 0:
        return 1
    elif attach_label == 1 and cc_count <= 0:
        return 3
    else:
        return 3
    
def get_dataset():
    try:
        df = pd.read_csv('../cc_attach_project.csv',index_col=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    
    to_drop = ['ENROLLMENT_DATE', 'DATE']
    df.drop(to_drop, inplace=True, axis=1)
    # df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
    # df['DATE'] = pd.to_datetime(df['DATE'])
    df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
    df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
    df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
    df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
    df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
    df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
    df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
    # df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
    cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
    df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
    df['INDUSTRY'] = df['INDUSTRY'].astype('category')
    df['ORGSIZE'] = df['ORGSIZE'].abs()
    # df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?

    #convert to all floats
    # df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

    #org_id same means same org!!!
#     weight = df.apply(assign_weight, axis=1)

#     df.insert(0, 'weight', weight, allow_duplicates=False) #insert weight into the df
    df = df.reindex(columns, axis='columns')
    df = pd.get_dummies(df)
    train, test = train_test_group_by_id(df, 0.8)
    val, test = train_test_group_by_id(test, 0.5)
    train.drop('ORGANIZATION_ID', inplace=True, axis=1)
    val.drop('ORGANIZATION_ID', inplace=True, axis=1)
    test.drop('ORGANIZATION_ID', inplace=True, axis=1)
    print('ORGANIZATION_ID' in train, 'ORGANIZATION_ID' in val, 'ORGANIZATION_ID' in test)
#     df = df.sample(frac=1, replace=False, random_state=1) #Shuffle the df
    train = train.sample(frac=1, random_state=123)
    val = val.sample(frac=1, random_state=123)
    test = test.sample(frac=1, random_state=123)
#     train, val, test = np.split(df.sample(frac=1), [int(.8 * len(df)), int(.9 * len(df))])
    
    print(train.shape, test.shape, val.shape)
    
    return (train, test, val)

def get_dataset_new():
    try:
        df = pd.read_csv('../cc_attach_project.csv',index_col=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    
    # df['ENROLLMENT_DATE'] = pd.to_datetime(df['ENROLLMENT_DATE'])
    # df['DATE'] = pd.to_datetime(df['DATE'])
    df['STAGENAME_PRODUCT'] = df['STAGENAME_PRODUCT'].fillna('not called')
    df['INDUSTRY'] = df['INDUSTRY'].fillna('missing')
    df['CASH_PERC'] = df['CASH_PERC'].fillna(0.0)  
    df['CHECK_PERC'] = df['CHECK_PERC'].fillna(0.0)
    df['OTHER_PERC'] = df['OTHER_PERC'].fillna(0.0)
    df['ACH_PERC'] = df['ACH_PERC'].fillna(0.0)
    df['CC_PERC'] = df['CC_PERC'].fillna(0.0)
    # df['BILLING_PLAN_ID'] = df['BILLING_PLAN_ID'].astype('int64')
    cat_dtype = CategoricalDtype(categories=(df['PLAN_TIER'].unique()), ordered=True)
    df['PLAN_TIER'] = df['PLAN_TIER'].astype(cat_dtype) #small > medium?
    df['INDUSTRY'] = df['INDUSTRY'].astype('category')
    df['ORGSIZE'] = df['ORGSIZE'].abs()
    # df['ORGSIZE'].replace(0,1, inplace=True) #should I do this?

    #convert to all floats
    # df = df.apply(pd.to_numeric, errors='ignore', downcast='float')

    #org_id same means same org!!!
    weight = df.apply(assign_weight, axis=1)

    df.insert(0, 'weight', weight, allow_duplicates=False) #insert weight into the df
    to_drop = ['ENROLLMENT_DATE', 'DATE', 'CC_COUNT']
    df.drop(to_drop, inplace=True, axis=1)

    df = df.reindex(columns_new, axis='columns')
    df = pd.get_dummies(df)
    train, test = train_test_group_by_id(df, 0.8)
    val, test = train_test_group_by_id(test, 0.5)
    train.drop('ORGANIZATION_ID', inplace=True, axis=1)
    val.drop(['ORGANIZATION_ID', 'weight'], inplace=True, axis=1)
    test.drop('ORGANIZATION_ID', inplace=True, axis=1)
    print('ORGANIZATION_ID' in train, 'ORGANIZATION_ID' in val, 'ORGANIZATION_ID' in test)
#     df = df.sample(frac=1, replace=False, random_state=1) #Shuffle the df
    train = train.sample(frac=1, random_state=123)
    val = val.sample(frac=1, random_state=123)
    test = test.sample(frac=1, random_state=123)
#     train, val, test = np.split(df.sample(frac=1), [int(.8 * len(df)), int(.9 * len(df))])
    
    print(train.shape, test.shape, val.shape)
    
    return (train, test, val)



In [28]:
# train, test, val = get_dataset()
# path = 'features_dummies.txt'
# file = open(path, 'w')
# with open(path, 'w') as writer:
#     for col in train.columns:
#         writer.write('{}{}'.format(col, '\n'))
# train.to_csv('../Feature-eng/{}.csv'.format(FILE_TRAIN), header=True, index=False)
# test.to_csv('../Feature-eng/{}.csv'.format(FILE_TEST), header=True, index=False)
# val.to_csv('../Feature-eng/{}.csv'.format(FILE_VALIDATION), header=True, index=False)

# print(train.head())
# print(train.columns)
# df = pd.read_csv('../cc_attach_project.csv',index_col=0, nrows=20000)
# df.columns
# test.to_csv('test.csv', header=False, index=False)

In [29]:

def write_to_s3(fobj, bucket, key):
    return boto3.Session(region_name=my_region).resource('s3').Bucket(bucket).Object(key).upload_file(fobj)

def upload_to_s3(partition_name, partition):
#     num_partition = 4                                 # partition file into 5 parts
#     partitions = np.array_split(partition, num_partition)
    filename = "./AWS-files/{}.csv".format(partition_name)
    partition.to_csv(filename, header=False, index=False)
    aws_filename = "{}.csv".format(partition_name)
    key = "{}/{}/{}".format(prefix,partition_name,aws_filename)
    url = 's3n://{}/{}'.format(bucket, key)
    print('Writing to {}'.format(url))
    write_to_s3(filename, bucket, key)
    print('Done writing to {}'.format(url))

def download_from_s3(partition_name, number, filename):
    key = "{}/{}/{}".format(prefix,partition_name, number)
    url = 's3n://{}/{}'.format(bucket, key)
    print('Reading from {}'.format(url))
    s3 = boto3.resource('s3', region_name = my_region)
    s3.Bucket(bucket).download_file(key, filename)
    try:
        s3.Bucket(bucket).download_file(key, 'mnist.local.test')
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print('The object does not exist at {}.'.format(url))
        else:
            raise        

def upload_data():
    train, test, val = get_dataset_new()
    partitions = [(FILE_TRAIN, train), (FILE_VALIDATION, val), (FILE_TEST, test)]
    for partition_name, partition in partitions:
        print('{}: {}'.format(partition_name, partition.shape))
        upload_to_s3(partition_name, partition)


In [30]:
upload_data()

Success: Data loaded into dataframe.
0
964500 964490
(964500, 171) (241112, 171)
0
120575 120556
(120575, 171) (120537, 171)
False False False
(964500, 170) (120537, 170) (120575, 169)
train: (964500, 170)
Writing to s3n://hcp-ml-project-tune-weight/xgboost/train/train.csv
Done writing to s3n://hcp-ml-project-tune-weight/xgboost/train/train.csv
validation: (120575, 169)
Writing to s3n://hcp-ml-project-tune-weight/xgboost/validation/validation.csv
Done writing to s3n://hcp-ml-project-tune-weight/xgboost/validation/validation.csv
test: (120537, 170)
Writing to s3n://hcp-ml-project-tune-weight/xgboost/test/test.csv
Done writing to s3n://hcp-ml-project-tune-weight/xgboost/test/test.csv


In [9]:
# partitions = [(FILE_TRAIN, train), (FILE_VALIDATION, val), (FILE_TEST, test)]
# for partition_name in [FILE_TRAIN, FILE_VALIDATION, FILE_TEST]:
# #     print('{}: {}'.format(partition_name, partition.shape))
#     filename = "{}.csv".format(partition_name)
#     key = "{}/{}/{}".format(prefix,partition_name,filename)
#     url = 's3n://{}/{}'.format(bucket, key)
#     print('Writing to {}'.format(url))
#     write_to_s3(filename, bucket, key)
#     print('Done writing to {}'.format(url))
